This notebook continuously estimates the prevailing, realtime, positivity values via 

* $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$

curves, wherein

$\qquad \qquad \qquad positives_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{P_{\delta}}  \right) \times \frac{1}{population}$


$\qquad \qquad \qquad \qquad  tests_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{T_{\delta}} \right) \times \frac{1}{population}$  

<br>

which are the positive cases per hundred thousand and test numbers per hundred thousand, respectively, based on their cumulative values and

<br>

<table style="width:35%; text-align: left; border: 0px solid black; float:left; margin-left: 50px">
    <tr>
        <th style="width:20%;">$\mathcal{Variable} \qquad$</th><th></th> 
    </tr>
    <tr>
        <td>$\tau$</td><td>end date</td>
    </tr>
    <tr>
        <td>$\delta$</td><td>a series date</td>
    </tr>
    <tr>
      <td>$P_{\delta}$</td><td>The number of positive cases on date $\delta$.</td>
    </tr>
    <tr>
      <td>$T_{\delta}$</td><td>The number of tests on date $\delta$.</td>
    </tr>
    
</table>


<br>

<br>

The subscript $c$ denotes cumulative.   The graph image below illustrates the curves in question.  In brief 

* The quotient of the coordinates of a point along a curve is the cumulative positivity value. 

* The gradient of a modelled secant line to a sequential set of curve rear points estimates the prevailing positivity value.

* The aforementioned secant line model predicts the expected values of $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}}$ increases.


<div style="margin-left: 100px;">
    <img src="https://github.com/briefings/sars/raw/develop/fundamentals/atlantic/docs/ptr.png" longdesc="surveillance" width="40%" style="float:center">
</div>

<br>

Upcoming: Link this image to the interactive graph.



<br>

## Preliminaries

### Libraries

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import dask

import logging

import os
import pathlib
import sys

import statsmodels.regression.linear_model


In [2]:
from statsmodels.formula.api import ols

<br>

### Paths

In [3]:
child = os.getcwd()
parent = str(pathlib.Path(child).parent)

In [4]:
root = os.path.join(child, 'warehouse')
warehouse = os.path.join(root, 'prospects')

<br>

Appending paths

In [5]:
sys.path.append(parent)

<br>

### Logging

In [6]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

<br>

### Custom

Import

In [7]:
import cartographs.boundaries.us.boundaries
import cartographs.boundaries.us.settings

In [8]:
import algorithms.curves.estimates
import algorithms.curves.predictions
import algorithms.curves.secants

In [9]:
import atlantic.base.directories
import atlantic.investigations.prospects
import atlantic.investigations.hospitalizations

<br>

Set-up directories

In [10]:
directories = atlantic.base.directories.Directories()
directories.cleanup(listof=[warehouse])
directories.create(listof=[warehouse])

<br>
<br>

### Classes

**Boundaries**

In [11]:
settings = cartographs.boundaries.us.settings.Settings()
boundaries = cartographs.boundaries.us.boundaries.Boundaries(settings.crs)

In [12]:
states = boundaries.states(year=settings.latest)
states.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   56 non-null     object  
 1   STATENS   56 non-null     object  
 2   AFFGEOID  56 non-null     object  
 3   GEOID     56 non-null     object  
 4   STUSPS    56 non-null     object  
 5   NAME      56 non-null     object  
 6   LSAD      56 non-null     object  
 7   ALAND     56 non-null     int64   
 8   AWATER    56 non-null     int64   
 9   geometry  56 non-null     geometry
dtypes: geometry(1), int64(2), object(7)
memory usage: 4.5+ KB


<br>
<br>

## Data

<br>

**Attributes**

In [13]:
fields = ['datetimeobject', 'STUSPS', 'positiveIncrease', 'testIncrease', 'deathIncrease', 'hospitalizedIncrease', 
          'positiveCumulative', 'testCumulative',  'deathCumulative', 'hospitalizedCumulative', 
          'positiveRate', 'testRate', 'deathRate', 'hospitalizedRate', 'ndays']

dtype = {'STUSPS': 'str', 'positiveIncrease': np.float64, 'testIncrease': np.float64, 'deathIncrease': np.float64, 
         'hospitalizedIncrease': np.float64, 'positiveCumulative': np.float64, 'testCumulative': np.float64, 
         'deathCumulative': np.float64, 'hospitalizedCumulative': np.float64, 'positiveRate': np.float64, 
         'testRate': np.float64, 'deathRate': np.float64, 'hospitalizedRate': np.float64, 'ndays': np.int64}        

parse_dates = ['datetimeobject']

<br>
<br>

**The URL strings of the sources**

In [14]:
uribaseline = os.path.join(parent, 'warehouse', 'baselines.csv')
urihospitalizations = os.path.join(parent, 'warehouse', 'hospitalizations.csv')

<br>
<br>

**Baseline**

In [15]:
baselines = pd.read_csv(filepath_or_buffer=uribaseline, header=0, usecols=fields, 
                        dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

baselines.loc[:, 'positiveTestRate'] = np.where(
    baselines['testRate'] > 0, 100 * baselines['positiveRate'] / baselines['testRate'], 0)
baselines.loc[:, 'deathPositiveRate'] = np.where(
    baselines['positiveRate'] > 0, 100 * baselines['deathRate'] / baselines['positiveRate'], 0)

logger.info(baselines.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17524 entries, 0 to 17523
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   datetimeobject          17524 non-null  datetime64[ns]
 1   STUSPS                  17524 non-null  object        
 2   deathIncrease           17524 non-null  float64       
 3   deathCumulative         17524 non-null  float64       
 4   positiveIncrease        17524 non-null  float64       
 5   positiveCumulative      17524 non-null  float64       
 6   hospitalizedIncrease    17524 non-null  float64       
 7   hospitalizedCumulative  17524 non-null  float64       
 8   testIncrease            17524 non-null  float64       
 9   testCumulative          17524 non-null  float64       
 10  deathRate               17524 non-null  float64       
 11  positiveRate            17524 non-null  float64       
 12  testRate                17524 non-null  float6

<br>
<br>

**Hospitalizations**

In [16]:
hospitalizations = pd.read_csv(filepath_or_buffer=urihospitalizations, header=0, 
                               usecols=fields,dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

hospitalizations.loc[:, 'hospitalizedPositiveRate'] =  np.where(hospitalizations['positiveRate'] > 0, 
                                                                100 * hospitalizations['hospitalizedRate'] / hospitalizations['positiveRate'], 0)
hospitalizations.loc[:, 'deathHospitalizedRate'] =  np.where(hospitalizations['hospitalizedRate'] > 0, 
                                                             100 * hospitalizations['deathRate'] / hospitalizations['hospitalizedRate'], 0)

logger.info(hospitalizations.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12469 entries, 0 to 12468
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   datetimeobject            12469 non-null  datetime64[ns]
 1   STUSPS                    12469 non-null  object        
 2   deathIncrease             12469 non-null  float64       
 3   deathCumulative           12469 non-null  float64       
 4   positiveIncrease          12469 non-null  float64       
 5   positiveCumulative        12469 non-null  float64       
 6   hospitalizedIncrease      12469 non-null  float64       
 7   hospitalizedCumulative    12469 non-null  float64       
 8   testIncrease              12469 non-null  float64       
 9   testCumulative            12469 non-null  float64       
 10  deathRate                 12469 non-null  float64       
 11  positiveRate              12469 non-null  float64       
 12  testRate          

<br>
<br>

## Regression

<br>

**Latest Date**

In [17]:
latestdate = baselines.datetimeobject.max()
logger.info(type(latestdate))
logger.info(latestdate)

INFO:__main__:<class 'pandas._libs.tslibs.timestamps.Timestamp'>
INFO:__main__:2020-12-23 00:00:00


<br>
<br>

**Periods**

Within the context of the aforementioned $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$ curves, each value of this array is the number of data points, starting from the end of a curve, that are used to develop **a regression model of the tangent/secant line to the points**.

Key points:

* The model predicts the expected $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}, \tau}$ increases.

* The gradient of each model is an estimate of the prevailing positivity rate; an estimate of the current daily positivity value. [cf. continous positivity value] 

In [18]:
periods = np.arange(8, 29)
logger.info( periods)

INFO:__main__:[ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28]


<br>
<br>

**Partitions**

In [19]:
partitionby = 'STUSPS'
partitions = baselines[partitionby].unique()
logger.info(partitions)

INFO:__main__:['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE'
 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY' 'PR']


<br>
<br>

### **positives vs. tests**

<br>

**Analysis**

In [20]:
formula = 'positiveRate ~ testRate'
regressor = 'testRate'

prospects = atlantic.investigations.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=100, num=5, latestdate=latestdate)

In [21]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

**State of affairs**

In [22]:
ptr = estimates

<br>

Prevailing Positive Test Rate

In [23]:
ptr.loc[:, 'prevailingPTR'] = 100 * ptr['gradient']
ptr = ptr.merge(baselines[['datetimeobject', 'STUSPS', 'positiveTestRate']], how='left', on=['datetimeobject', 'STUSPS'])
ptr.loc[:, 'cf'] = ptr[['prevailingPTR', 'positiveTestRate']].max(axis=1)

ptr.loc[:, 'rank'] = ptr['cf'].rank(method='min', ascending=False)
ptr.sort_values(by='rank', ascending=True, inplace=True)
ptr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [24]:
ptr.loc[:, 'prevailingTPC'] = np.where(ptr['prevailingPTR'] > 0, ptr['prevailingPTR'].rdiv(100), 0)
ptr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 
                    'positiveTestRate': 'continuousPTR'},
          inplace=True)
ptr.to_csv(path_or_buf=os.path.join(warehouse, 'ptr.csv'), index=False, header=True, encoding='utf-8')
ptr.to_json(path_or_buf=os.path.join(warehouse, 'ptr.json'), orient='values')

ptr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingPTR,continuousPTR,rank,prevailingTPC
37,2020-12-23,0.0535354,0.0142052,0.0719972,4.15325e-09,0.96088,-673.826,OR,5.35354,4.208673,48.0,18.6792
32,2020-12-23,0.0528087,0.0505482,0.055399,1.1118e-28,0.999559,-1966.59,NY,5.28087,3.685228,49.0,18.9363
1,2020-12-23,0.0525285,0.0375105,0.0620592,1.57514e-18,0.992037,-2798.94,AK,5.25285,3.548285,50.0,19.0373
19,2020-12-23,0.0520382,0.0383356,0.0704868,1.12824e-15,0.980804,-2599.15,ME,5.20382,1.935673,51.0,19.2166
8,2020-12-23,0.0354076,0.0297355,0.0380092,1.05139e-20,0.997179,-389.132,DC,3.54076,3.203089,52.0,28.2425


<br>
<br>

### **deaths vs. positives**

In [25]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [26]:
formula = 'deathRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.investigations.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=20, num=5, latestdate=latestdate)

In [27]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

**State of Affairs**

In [28]:
dpr = estimates

<br>

Prevailing Death Positive Rate

In [29]:
dpr.loc[:, 'prevailingDPR'] = 100 * dpr['gradient']
dpr = dpr.merge(baselines[['datetimeobject', 'STUSPS', 'deathPositiveRate']], how='left', on=['datetimeobject', 'STUSPS'])
dpr.loc[:, 'cf'] = dpr[['prevailingDPR', 'deathPositiveRate']].max(axis=1)

dpr.loc[:, 'rank'] = dpr['cf'].rank(method='min', ascending=False)
dpr.sort_values(by='rank', ascending=True, inplace=True)
dpr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [30]:
dpr.loc[:, 'prevailingCPD'] = np.where(dpr['prevailingDPR'] > 0, dpr['prevailingDPR'].rdiv(100), 0)
dpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 
                    'deathPositiveRate': 'continuousDPR'},
          inplace=True)
dpr.to_csv(path_or_buf=os.path.join(warehouse, 'dpr.csv'), index=False, header=True, encoding='utf-8')
dpr.to_json(path_or_buf=os.path.join(warehouse, 'dpr.json'), orient='values')

dpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingDPR,continuousDPR,rank,prevailingCPD
42,2020-12-23,0.00988069,0.00879089,0.0131188,1.67585e-18,0.992651,14.129,TN,0.988069,1.178775,48.0,101.207
17,2020-12-23,0.00807577,0.00699802,0.0110684,1.70826e-15,0.987958,9.54415,KY,0.807577,0.985296,49.0,123.827
36,2020-12-23,0.00698064,0.00490129,0.00753129,7.06989e-18,0.991425,10.0607,OK,0.698064,0.847829,50.0,143.253
1,2020-12-23,0.00629447,0.00241803,0.0104046,2.46292e-11,0.941798,-10.8379,AK,0.629447,0.454325,51.0,158.87
44,2020-12-23,0.00574828,0.00425343,0.00646154,3.06088e-19,0.9874,-8.94956,UT,0.574828,0.464111,52.0,173.965


<br>
<br>

### **hospitalizations vs. positives**

In [31]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [32]:
formula = 'hospitalizedRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.investigations.prospects.Prospects(data=hospitalizations, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=10, num=5, latestdate=latestdate)

In [33]:
estimates, predictions = prospects.exc(partitions=hospitalizations[partitionby].unique())

<br>

**State of Affairs**

In [34]:
hpr = estimates

<br>

Prevailing Hospitalised Positive Rate

In [35]:
hpr.loc[:, 'prevailingHPR'] = 100 * hpr['gradient']
hpr.loc[:, 'rank'] = hpr['prevailingHPR'].rank(method='min', ascending=False)
hpr.sort_values(by='rank', ascending=True, inplace=True)

<br>

Structuring

In [36]:
hpr = hpr.merge(hospitalizations[['datetimeobject', 'STUSPS', 'hospitalizedPositiveRate']], how='left', on=['datetimeobject', 'STUSPS'])
hpr.loc[:, 'prevailingCPH'] = np.where(hpr['prevailingHPR'] > 0, hpr['prevailingHPR'].rdiv(100), 0)

hpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'hospitalizedPositiveRate': 'continuousHPR'}, 
           inplace=True)

hpr.to_csv(path_or_buf=os.path.join(warehouse, 'hpr.csv'), index=False, header=True, encoding='utf-8')
hpr.to_json(path_or_buf=os.path.join(warehouse, 'hpr.json'), orient='values')

hpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingHPR,rank,continuousHPR,prevailingCPH
32,2020-12-23,0.0130372,0.00251821,0.0201443,8.53633e-08,0.841278,182.78,MS,1.30372,33.0,4.012979,76.7034
33,2020-12-23,0.0093742,0.00754853,0.0113468,2.79488e-18,0.991376,130.151,TN,0.93742,34.0,2.581110,106.676
34,2020-12-23,0.00363039,0.00241727,0.00591931,1.32871e-11,0.942074,55.2721,NH,0.363039,35.0,2.316161,275.452
35,2020-12-23,3.46945e-18,-5.29631e-15,5.29631e-15,0.621669,-20.4737,343.787,CT,3.46945e-16,36.0,7.180223,2.8823e+17
36,2020-12-23,-1.38778e-17,-1.21092e-14,1.1443e-14,0.829481,-90.6731,462.615,NY,-1.38778e-15,36.0,10.241811,0


<br>
<br>

### **deaths vs. hospitalizations**

In [37]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [38]:
formula = 'deathRate ~ hospitalizedRate'
regressor = 'hospitalizedRate'

prospects = atlantic.investigations.prospects.Prospects(data=hospitalizations, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=10, num=5, latestdate=latestdate)

In [39]:
estimates, predictions = prospects.exc(partitions=hospitalizations[partitionby].unique())

<br>

**State of Affairs**

In [40]:
dhr = estimates

<br>

Prevailing

In [41]:
dhr.loc[:, 'prevailingDHR'] = 100 * dhr['gradient']
dhr.loc[:, 'rank'] = dhr['prevailingDHR'].rank(method='min', ascending=False)
dhr.sort_values(by='rank', ascending=True, inplace=True)

<br>

Structuring

In [42]:
dhr = dhr.merge(hospitalizations[['datetimeobject', 'STUSPS', 'deathHospitalizedRate']], how='left', on=['datetimeobject', 'STUSPS'])
dhr.loc[:, 'prevailingHPD'] = np.where(dhr['prevailingDHR'] > 0, dhr['prevailingDHR'].rdiv(100), 0)

dhr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'deathHospitalizedRate': 'continuousDHR'}, 
           inplace=True)

dhr.to_csv(path_or_buf=os.path.join(warehouse, 'dhr.csv'), index=False, header=True, encoding='utf-8')
dhr.to_json(path_or_buf=os.path.join(warehouse, 'dhr.json'), orient='values')

dhr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingDHR,rank,continuousDHR,prevailingHPD
32,2020-12-23,0.16104,0.110455,0.180269,3.26762e-18,0.988558,-14.5021,UT,16.104,33.0,11.581292,6.20962
33,2020-12-23,0.154251,0.0954782,0.285958,3.32073e-09,0.895922,13.3788,WA,15.4251,34.0,23.038999,6.48293
34,2020-12-23,0.153826,0.13335,0.1737,8.0891e-21,0.99687,-3.46686,OK,15.3826,35.0,14.561806,6.50084
35,2020-12-23,0.132913,0.0820883,0.211224,1.28227e-11,0.946845,6.49772,AL,13.2913,36.0,14.492433,7.52375
36,2020-12-23,0.0417448,0.00323372,0.105122,7.93051e-06,0.751772,15.0173,HI,4.17448,37.0,16.735173,23.9551


<br>
<br>

## Clean-up

In [43]:
!rm -rf *.log
!rm -rf *.pdf
!rm -rf states

<br>
<br>

## End

In [44]:
%%bash

date +"%Y-%m-%d %T"

2020-12-24 17:14:43
